In [4]:
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import tweepy
import json
import os
from collections import Counter
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt
import time

consumer_key="hME3af4IKB8PG1bACoFWIOsb7"
consumer_secret="5NVgAZqgezRM1fApMwuv4bF2lagForppzEClipxYymL4EnWztb"
access_token="2257981634-ERJNYPvjln32aQlNygNAyUNImFn2TCzE1VBYIkG"
access_token_secret="hqmDnz3x0iQoEbmAJvq6AAN1rFHbuFetoTs5hPf20Gqdq"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [8]:
# funciton to get tweet ids already in file
def get_ids(file_name):
    result=[]
    try:
        with open(file_name, 'r') as fp:
            line=fp.readline()
            while line:
                tjson=json.loads(line)
                result.append(tjson["id_str"])
                line=fp.readline()
        return result
    except IOError:
        return result

# call get_ids on starting file
tweet_ids = get_ids('tweets.json')

# function from tutorial
def crawl_tweet(file_name="raw_tweet.json", search_key="Twitter", max_tweet=1000, max_id=-1):
    tweet_count = 0
    # append to file instead of writing so we don't wipe previous data
    with open(file_name, 'a') as fp:
        fp.write("")
    while tweet_count<max_tweet:
        try:
            if max_id<=0:
                new_tweets=api.search(q=search_key, count=100)
            else:
                new_tweets=api.search(q=search_key, count=100, max_id=str(max_id-1))
            if not new_tweets:
                print('No more tweets')
                break
            with open(file_name, 'a') as fp:
                for tweet in new_tweets:
                    # convert tweet to json
                    tjson=tweet._json
                    # filter duplicate tweets
                    if tjson['id_str'] not in tweet_ids:
                        # add tweet to text file
                        fp.write(json.dumps(tjson)+"\n")
                        # add tweet id to tweet_ids
                        tweet_ids.append(tjson['id_str'])
            max_id=new_tweets[-1].id
            tweet_count+=len(new_tweets)
            file_name.close()
            print(f"Number of tweets: {tweet_count}")
        except tweepy.TweepError as e:
            print("some error: " + str(e))
            break
        


In [6]:
### read keywords

input_keywords="input_keywords.txt"
# read keyswords in
with open(input_keywords) as fp2:
    keywords = fp2.readlines()
    
# remove newlines
for i in range(len(keywords)):
    keywords[i] = keywords[i].replace('\n', '')

In [7]:
### Crawl tweets from most recent

# call crawel tweets on each keyword
for k in keywords:
    crawl_tweet(file_name='tweets.json', search_key=k, count=100)

Number of tweets: 100
Number of tweets: 200
Number of tweets: 300
Number of tweets: 400
Number of tweets: 500
Number of tweets: 600
Number of tweets: 700
Number of tweets: 800
Number of tweets: 900
Number of tweets: 1000
Number of tweets: 100
Number of tweets: 200
Number of tweets: 299
Number of tweets: 399
Number of tweets: 499
Number of tweets: 599
Number of tweets: 699
Number of tweets: 799
Number of tweets: 899
Number of tweets: 999
Number of tweets: 1099
Number of tweets: 100
Number of tweets: 199
Number of tweets: 299
Number of tweets: 399
Number of tweets: 499
Number of tweets: 599
Number of tweets: 699
Number of tweets: 799
Number of tweets: 883
Number of tweets: 983
Number of tweets: 1083
Number of tweets: 100
Number of tweets: 191
Number of tweets: 290
Number of tweets: 390
Number of tweets: 459
Number of tweets: 551
Number of tweets: 635
Number of tweets: 735
Number of tweets: 835
Number of tweets: 935
Number of tweets: 1035


In [ ]:
### crawl tweets from oldest stored

# call get_ids on starting file
tweet_ids = get_ids('tweets.json')
last = tweet_ids.min()
# call crawel tweets on each keyword
for k in keywords:
    crawl_tweet('tweets.json', k, 10000, last)